# Chapter 6 Enginereing Sustainability: Evapotranspiraiton

1. [Introduction: Evapotranspiration](#1.-Introduction)
2. [Simulation: Evapotranspiration](#2.-Simulation)
3. [Self-Assessment](#3.-Self-Assessment)

## 1. Introduction

### 🌿 Theoretical Background: Potential Evapotranspiration (PET) Models

Evapotranspiration (ET) represents the combined loss of water from soil evaporation and plant transpiration. Estimating **potential evapotranspiration (PET)** is essential for hydrological modeling, irrigation planning, and climate impact assessment. The models below vary in complexity, data requirements, and physical assumptions.

---

### 🔀 Mixed-Method Models

### 1. **Penman (1948)**
Combines energy balance and aerodynamic principles. Requires temperature, radiation, humidity, and wind speed.

**Equation:**
$$
ET = \frac{\Delta R_n + \gamma \frac{900}{T + 273} u (e_s - e_a)}{\Delta + \gamma (1 + 0.34u)}
$$
- $( R_n $): net radiation (MJ/m²/day)  
- $( T $): mean air temperature (°C)  
- $( u $): wind speed (m/s)  
- $( e_s $), $( e_a $): saturation and actual vapor pressure (kPa)  
- $( \Delta $): slope of vapor pressure curve  
- $( \gamma $): psychrometric constant

### 2. **Turc (1961)**
Empirical model using temperature, solar radiation, and relative humidity.

**Equation:**
$$
ET = \frac{0.013 \cdot (T / (T + 15)) \cdot (Rs + 50)}{\sqrt{RH / 100}}
$$
- $( T $): mean temperature (°C)  
- $( Rs $): solar radiation (MJ/m²/day)  
- $( RH $): relative humidity (%)


### 🌞 Radiation-Based Models

### 3. **Priestley-Taylor (1972)**
Simplified energy balance model assuming minimal aerodynamic influence.

**Equation:**
$$
ET = \alpha \cdot \frac{\Delta}{\Delta + \gamma} \cdot R_n
$$
- $( \alpha $): empirical coefficient (~1.26)  
- $( R_n $): net radiation  
- $( \Delta $), $( \gamma $): as above

### 4. **Makkink (1957)**
Uses temperature and radiation, calibrated for temperate climates.

**Equation:**
$$
ET = 0.61 \cdot \frac{\Delta}{\Delta + \gamma} \cdot Rs
$$
- $( Rs $): solar radiation  
- $( \Delta $), $( \gamma $): as above

---

### 🌡️ Temperature-Based Models

### 5. **Thornthwaite (1948)**
Empirical model based solely on temperature and latitude.

**Equation:**
$$
ET = 16 \cdot \left(\frac{10T}{I}\right)^a
$$
- $( T $): mean monthly temperature (°C)  
- $( I $): annual heat index  
- $( a $): empirical exponent based on $( I $)

### 6. **Hamon (1963)**
Estimates ET using temperature and day length.

**Equation:**
$$
ET = 0.1651 \cdot D \cdot \frac{e_s}{T + 273.2}
$$
- $( D $): day length (hours)  
- $( e_s $): saturation vapor pressure

---

### 📉 Temperature-Radiation Hybrid Models

### 7. **Hargreaves-Samani (1985)**
Uses temperature range and extraterrestrial radiation.

**Equation:**
$$
ET = 0.0023 \cdot (T_{mean} + 17.8) \cdot (T_{max} - T_{min})^{0.5} \cdot R_a
$$
- $( R_a $): extraterrestrial radiation  
- $( T_{mean} $), $( T_{max} $), $( T_{min} $): temperatures

---

### 🧠 Simplified Empirical Model

### 8. **Oudin (2005)**
Minimalist model using temperature and latitude.

**Equation:**
$$
ET = \frac{Rs}{100} \cdot \left(\frac{T + 5}{100}\right)
$$
- Often reformulated using temperature and latitude-based radiation estimates.

---

## 🧪 Model Selection Considerations

| Model                | Inputs Required                     | Complexity | Best Use Case                      |
|---------------------|--------------------------------------|------------|------------------------------------|
| Penman              | T, RH, Rs, wind                      | High       | Detailed hydrological modeling     |
| Priestley-Taylor    | T, Rs                                | Medium     | Energy-dominated environments      |
| Hargreaves-Samani   | T_max, T_min, latitude               | Medium     | Data-sparse regions                |
| Thornthwaite        | T, latitude                          | Low        | Long-term climate analysis         |
| Turc                | T, Rs, RH                            | Medium     | Mediterranean climates             |
| Makkink             | T, Rs                                | Medium     | Temperate zones                    |
| Hamon               | T, latitude                          | Low        | Simpler water balance models       |
| Oudin               | T, latitude                          | Very Low   | Quick screening or large datasets  |

---

## 2. Simulation

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import pyet

# 📥 Load and preprocess data

# data = pd.read_csv("WR_alexandria.csv", parse_dates=True, index_col="date")
# data = pd.read_csv("WR_alexandria.csv", parse_dates=True, index_col="date")

# # 🌤️ Meteorological inputs
# meteo = pd.DataFrame({
#     "tmax": data["tmax"],
#     "tmin": data["tmin"],
#     "rain": data["rain"],
#     "umax": data["maxu2"] * 0.447,  # convert to m/s
#     "u": data["u2"] * 0.447,
#     "rhmax": data["rhmax"] / 100,
#     "rhmin": data["rhmin"] / 100,
#     "sol": data["solar"],
#     "tsmax": data["tsmax"],
#     "tsmin": data["tsmin"]
# })


# 📥 Read the tab-separated file
#df = pd.read_csv('WR_Alexandria.csv', parse_dates = True, parse_dates=['date'])
#df = pd.read_csv('WR_Alexandria.csv', sep='\t', parse_dates=['date'])

df = pd.read_csv('WR_Alexandria.csv', sep=',', parse_dates=['date'])
#with open('WR_Alexandria.csv', 'r') as f:
#    print(f.readline())
#date,tmax,tmin,rain,maxu2,u2,u2dir,rhmax,rhmin,solar,tsmax,tsmin

# 🧼 Clean and convert numeric columns
numeric_cols = df.columns.drop('date')  # all except date

for col in numeric_cols:
    # Strip whitespace, convert to numeric, coerce errors to NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 🧹 Optional: Handle missing values (if any)
df.fillna(method='ffill', inplace=True)  # forward fill
# Or: df.dropna(inplace=True)

# ✅ Display summary
print(df)
#print("\nData types:\n", df.dtypes)

# 🌍 Site parameters
lat = 39 * 3.1416 / 180  # radians
elevation = 10  # meters

# 🌡️ Derived variables
tmean = (data["tmax"] + data["tmin"]) / 2.0
rh = (data["rhmax"] + data["rhmin"]) / 2.0
sol = data["solar"]
tmax = data["tmax"]
tmin = data["tmin"]
u = data["u2"] * 0.447  # wind speed in m/s
#print(tmean)
#plt.plot(u)
# 🧮 Evapotranspiration Models

# 🔀 Mixed-methods
#et_penman = pyet.penman(tmean, u, rs=sol, elevation=elevation, lat=lat, tmax=tmax, tmin=tmin, rh=rh)
et_oudin = pyet.oudin(tmean, lat)
#et_makkink = pyet.makkink(tmean, rs=sol, elevation=elevation)
et_turc = pyet.turc(tmean, rs=sol, rh=rh)

# 🌞 Radiation-based
#et_priestley_taylor = pyet.priestley_taylor(tmean, rs=sol, elevation=elevation)
#et_hargreaves = pyet.hargreaves(tmean, tmax, tmin, lat)

# 🌡️ Temperature-based
#et_thornthwaite = pyet.thornthwaite(tmean, lat)
#et_hamon = pyet.hamon(tmean, lat)

# 📊 Plot comparison
plt.figure(figsize=(12, 6))
et_penman.plot(label="Penman", color="blue")
et_priestley_taylor.plot(label="Priestley-Taylor", color="green")
et_hargreaves.plot(label="Hargreaves-Samani", color="orange")
et_thornthwaite.plot(label="Thornthwaite", color="purple")
et_hamon.plot(label="Hamon", color="brown")
et_oudin.plot(label="Oudin", color="gray")
et_makkink.plot(label="Makkink", color="teal")
et_turc.plot(label="Turc", color="red")

plt.title("Evapotranspiration Estimates — Multiple Models")
plt.ylabel("ET (mm/day)")
plt.xlabel("Date")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'WR_Alexandria.csv'

## 3. Self-Assessment

### 🌿 Reflective, Conceptual, and Quiz Questions: Potential Evapotranspiration (PET) Models

---

### 📘 Conceptual Questions

These questions explore the physical principles, assumptions, and structure of the PET models.

### General Concepts
- What is the difference between actual evapotranspiration (ET) and potential evapotranspiration (PET)?
- Why is PET a critical variable in hydrology, agriculture, and climate modeling?
- How do meteorological inputs like temperature, radiation, humidity, and wind influence PET?

### Model Structure
- What distinguishes mixed-method models (e.g., Penman) from purely empirical or temperature-based models?
- Why do radiation-based models often assume minimal aerodynamic influence?
- How does latitude affect temperature-based models like Thornthwaite and Hamon?

### Data Requirements
- Which models are best suited for data-scarce environments, and why?
- How does the complexity of a model relate to its input requirements and accuracy?
- What are the trade-offs between using a simplified model like Oudin versus a physically-based model like Penman?

---

### 🔍 Reflective Questions

These questions encourage critical thinking and application to real-world modeling decisions.

- If you were modeling PET for a remote watershed with limited data, which model would you choose and why?
- How might climate change (e.g., increased temperature, altered radiation patterns) affect the reliability of temperature-based PET models?
- What are the implications of overestimating PET in irrigation planning or drought forecasting?
- How could you validate the performance of different PET models using observed ET data?
- In what scenarios might a medium-complexity model like Hargreaves-Samani outperform a high-complexity model like Penman?

---

### ❓ Quiz Questions

### Multiple Choice

1. **Which PET model requires the most meteorological inputs?**  
   A. Oudin  
   B. Hamon  
   C. Penman  
   D. Thornthwaite  
   **Answer:** C

2. **Which model is most appropriate for long-term climate analysis using only temperature and latitude?**  
   A. Priestley-Taylor  
   B. Thornthwaite  
   C. Turc  
   D. Makkink  
   **Answer:** B

3. **In the Hargreaves-Samani model, which variable represents extraterrestrial radiation?**  
   A. \( R_s \)  
   B. \( R_n \)  
   C. \( R_a \)  
   D. \( e_s \)  
   **Answer:** C

### True/False

4. **The Priestley-Taylor model assumes minimal influence from wind and humidity.**  
   **Answer:** True

5. **The Oudin model requires solar radiation and wind speed as inputs.**  
   **Answer:** False

6. **All PET models estimate actual evapotranspiration under field conditions.**  
   **Answer:** False

### Short Answer

7. **Explain why the Penman model is considered physically rigorous.**  
   *Answer:* It integrates both energy balance and aerodynamic components, accounting for radiation, temperature, humidity, and wind speed.

8. **What are the limitations of using temperature-only models like Thornthwaite in arid regions?**  
   *Answer:* They may underestimate PET due to lack of radiation and humidity inputs, which are critical in dry climates.

---